In [4]:
import torch
import os
import numpy as np
import pandas as pd
import imgaug.augmenters as iaa
import cv2
from kornia import tensor_to_image
from sklearn.model_selection import train_test_split
import sys

sys.path.append('../')

from utils.transforms import random_edge_homography, dictListToAugment
from datasets import ImagePairHomographyDataset
from utils.io import load_yaml

In [5]:
server = load_yaml('../configs/servers.yml')['local']
configs = load_yaml('../configs/deep_image_homography_estimation.yml')

prefix = os.path.join(server['database']['location'], configs['data']['pkl_path'])
df = pd.read_pickle(os.path.join(
        prefix,
        configs['data']['pkl_name']
))
rho = 32
crp_shape = [480, 640]

transforms = dictListToAugment(configs['data']['train_transforms'])

ds = ImagePairHomographyDataset(df, prefix, rho, crp_shape, transforms=transforms, seeds=np.arange(len(df)).tolist())

In [3]:
for idx, batch in enumerate(ds):
    img = tensor_to_image(batch['img_crp'])[...,::-1]
    wrp = tensor_to_image(batch['wrp_crp'])[...,::-1]

    cv2.imshow('img', img)
    cv2.imshow('wrp', wrp)
    cv2.waitKey()

In [10]:
# augment single image as example
transforms = dictListToAugment(configs['data']['train_transforms'])

# read first image
img = cv2.imread(os.path.join(prefix, df['path'][0], df['file_seq'][0][0]))

max_img = 100

for i in range(max_img):
    print('\r{}/{}'.format(i+1, max_img), end='')
    trf = transforms(img)

    cv2.imwrite('out/data_augmentation/transformed_img_{}.jpg'.format(i), trf)  # save output
cv2.destroyAllWindows()

100/100